In [23]:
import numpy as np
import pandas as pd
import datetime as dt
from gurobipy import *
from BOM_graph.StudyBOM import GenerateGraph
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# Definicion de los parametros del entorno gurobi

#Dado el número de variables de este modelo, es necesario utilizar gurobi con una licencia académica (se obtiene gratuitamente en la web de gurobi)
params = {"WLSACCESSID" : '5cbfde7e-b7bd-44e3-9e2d-fb335f5a2deb', "WLSSECRET" : 'f526471f-ed2c-462d-a933-1241228cd704', "LICENSEID" : 2489204 }
env = Env(params = params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2489204
Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es


In [11]:
# Definición de los parámetros del problema
BOM = pd.read_pickle('./DataFiles/BOM.pkl')
BOM = BOM[BOM['MyBOMITEMID'] != 44].reset_index(drop=True) # Delete non connected element
MixedItems = pd.read_pickle('./DataFiles/MixedItems.pkl')
MixedItems = MixedItems[MixedItems['MyBOMITEMID'] != 44].reset_index(drop=True) # Delete non connected element
PurchaseItems = pd.read_pickle('./DataFiles/PurchaseItems.pkl')
RouteItems = pd.read_pickle('./DataFiles/RouteItems.pkl')
Orders = pd.read_pickle('./DataFiles/Orders.pkl')
Orders = Orders[Orders['MyBOMITEMID'] != 44].reset_index(drop=True)
Orders['END_DATE'] = pd.to_datetime(Orders['END_DATE'])
Stock = pd.read_pickle('./DataFiles/Stock.pkl')
Stock = Stock[Stock['MyBOMITEMID'] != 44].reset_index(drop=True)

# Grafo
G = GenerateGraph(BOM, typeG_ND = False, connected = True)

# Items
NN = sorted(G.nodes)
NN_len = sorted(NN)
K1 = sorted(RouteItems["MyBOMITEMID"].unique().tolist())
K2 = sorted(PurchaseItems["MyBOMITEMID"].unique().tolist())
K3 = sorted(MixedItems["MyBOMITEMID"].unique().tolist())

# Arcos 
arcos = sorted(G.edges)

# Layers
layers_dict = {}
for node in G.nodes:
    layer = G.nodes[node]["layer"]
    if layer not in layers_dict:
        layers_dict[layer] = []
    layers_dict[layer].append(node)
layers = [sorted(nodes) for layer, nodes in sorted(layers_dict.items())]

# Conjuntos N(i) e inversos
N = {}
# Encontrar los ítems necesarios para cada ítem i en Kt ∪ K3
for i in sorted(K1+K3):
    N[i] = [j for _, j in G.edges(i)]

N_reverse = {j: [] for j in NN}
for j, neighbors in N.items():
    for i in neighbors:
        N_reverse[i].append(j)

# Clientes
R = sorted(Orders["CUSTOMERID"].unique().tolist())
R_len = len(R)

# Periodos de tiempo en el horizonte temporal
date_range = pd.date_range(start= dt.datetime.now().strftime("%Y-%m-01"), periods=13, freq='MS')
T = date_range.tolist() # De hoy a 12 meses

# Demanda y precios de venta
# Map MyBOMITEMID and CUSTOMERID to indices
item_indices = {item: idx for idx, item in enumerate(layers[0]+[39])} # El 39 es un ítem a nivel 0 y Layer 1
customer_indices = {customer: idx for idx, customer in enumerate(R)}
D = []
B = []
for period_start in T:
    period_end = period_start + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # End of the month
    period_df = Orders[(Orders['END_DATE'] >= period_start) & (Orders['END_DATE'] <= period_end)]
    
    # Create matrices for quantities and prices
    period_matrixD = np.zeros((len(layers[0]+[39]), R_len))
    period_matrixB = np.zeros((len(layers[0]+[39]), R_len))
    
    # Fill the matrices
    for _, row in period_df.iterrows():
        item_idx = item_indices[row['MyBOMITEMID']]
        customer_idx = customer_indices[row['CUSTOMERID']]
        period_matrixD[item_idx, customer_idx] = row['QUANTITY']
        period_matrixB[item_idx, customer_idx] = row['UNITPRICE_EUR']
    
    # Append the period matrices to the lists
    D.append(period_matrixD)
    B.append(period_matrixB)
    
# Costes 
c1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["RUNTIME_COST"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["RUNTIME_COST"]))}
c2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["UNITPRICE_Compra"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["UNITPRICE_Compra"]))}

# MOQs
MOQ1 = {**dict(zip(RouteItems["MyBOMITEMID"], RouteItems["MOQ_Fabricacion"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Fabricacion"]))}
MOQ2 = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["MOQ_Compra"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["MOQ_Compra"]))}

# Lead times
lt = {**dict(zip(PurchaseItems["MyBOMITEMID"], PurchaseItems["LEADTIME"])), 
  **dict(zip(MixedItems["MyBOMITEMID"], MixedItems["LEADTIME"]))}

# Stock
I_0 = dict(zip(Stock["MyBOMITEMID"], Stock["STOCK"]))


# Matriz alpha
# Construir la matriz alpha
alpha = {}

for i in sorted(K1 + K3):
    alpha[i] = {}
    for j in N[i]:
        maxibo_qty = BOM[(BOM['MyPARENTBOMITEMID'] == i) & (BOM['MyBOMITEMID'] == j)]['MAXIBOQTY']
        if not maxibo_qty.empty:
            alpha[i][j] = maxibo_qty.values[0]
        else:
            alpha[i][j] = 0

a


In [12]:
# Inicialización del modelo
modelo = Model("Ejercicio", env = env)

In [13]:
# Definición de las variables
indices_x = [(i,t) for t in range(1,len(T)+1) for i in K1+K3]
indices_z1 = [(i,t) for t in range(1,len(T)+1)  for i in K1+K3]
indices_y = [(i,t) for t in range(1,len(T)+1)  for i in K2+K3]
indices_z2 = [(i,t) for t in range(1,len(T)+1)  for i in K2+K3]
indices_w = [(i, r, t) for t in range(1,len(T)+1)  for i in layers[0]+[39] for r in R]
indices_I = [(i,t) for t in range(1,len(T)+1)  for i in NN]

x = modelo.addVars(indices_x, lb = 0,vtype = GRB.INTEGER, name = "x") 
z1 = modelo.addVars(indices_z1, lb = 0,vtype = GRB.BINARY, name = "z1")
y = modelo.addVars(indices_y, lb = 0, vtype = GRB.INTEGER, name = "y")
z2 = modelo.addVars(indices_z2, lb = 0,vtype = GRB.BINARY, name = "z2")
w = modelo.addVars(indices_w, lb = 0, vtype= GRB.BINARY, name = "w") 
It =  modelo.addVars(indices_I, lb = 0,vtype= GRB.INTEGER, name = "It")

modelo.update()

In [14]:
# Inventario para el primer periodo, para items a nivel 0
r10 = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(D[0][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K1).intersection(set(layers[0] + [39])))
    ),name="R10" )
r20 = modelo.addConstrs(
    (It[i,1] == I_0[i] + y[i, 1] - quicksum(D[0][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K2).intersection(set(layers[0] + [39]))) if lt[i]< 1
    ),name="R20" )
r30 = modelo.addConstrs(
    (It[i,1] == I_0[i] - quicksum(D[0][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K2).intersection(set(layers[0] + [39]))) if lt[i]>= 1
    ),name="R30" )
r40 = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] + y[i, 1] - quicksum(D[0][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K3).intersection(set(layers[0] + [39]))) if lt[i]< 1
    ),name="R40" )
r50 = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(D[0][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R) for i in list(set(K3).intersection(set(layers[0] + [39]))) if lt[i]>= 1
    ),name="R50" )

# Inventario para el primer periodo, para items a otro nivel
r1a = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K1).intersection(set().union(*layers[1:])))
    ),name="R1a" )
r2a = modelo.addConstrs(
    (It[i,1] == I_0[i] + y[i, 1] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K2).intersection(set().union(*layers[1:])  )) if lt[i]< 1
    ),name="R2a" )
r3a = modelo.addConstrs(
    (It[i,1] == I_0[i] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K2).intersection(set().union(*layers[1:])  )) if lt[i]>= 1
    ),name="R3a" )
r4a = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] + y[i, 1] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K3).intersection(set().union(*layers[1:])  )) if lt[i]< 1
    ),name="R4a" )
r5a = modelo.addConstrs(
    (It[i,1] == I_0[i] + x[i, 1] - quicksum(alpha[j][i]*x[j,1] for j in N_reverse[i]) for i in list(set(K3).intersection(set().union(*layers[1:])  )) if lt[i]>= 1
    ),name="R5a" )

# Inventario para el resto de periodos, para items a nivel 0
r10t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] - quicksum(D[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K1).intersection(set(layers[0] + [39]))) for t in range(2, len(T)+1)
    ),name="R10t" )
r20t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + y[i, t] - quicksum(D[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K2).intersection(set(layers[0] + [39]))) for t in range(2, len(T)+1) if lt[i]< t
    ),name="R20t" )
r30t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] - quicksum(D[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K2).intersection(set(layers[0] + [39]))) for t in range(2, len(T)+1) if lt[i]>= t
    ),name="R30t" )
r40t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t]  + y[i, t] - quicksum(D[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K3).intersection(set(layers[0] + [39]))) for t in range(2, len(T)+1) if lt[i]< t
    ),name="R40t" )
r50t = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t]  - quicksum(D[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R) for i in list(set(K3).intersection(set(layers[0] + [39]))) for t in range(2, len(T)+1) if lt[i]>= t
    ),name="R50t" )

# Inventario para el resto de periodos, para items a otro nivel
r1a = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K1).intersection(set().union(*layers[1:]))) for t in range(2, len(T)+1)
    ),name="R1a" )
r2a = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + y[i, t] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K2).intersection(set().union(*layers[1:]))) for t in range(2, len(T)+1) if lt[i]< 1
    ),name="R2a" )
r3a = modelo.addConstrs(
    (It[i,t] == It[i, t-1] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K2).intersection(set().union(*layers[1:]))) for t in range(2, len(T)+1) if lt[i]>= 1
    ),name="R3a" )
r4a = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] + y[i, t] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K3).intersection(set().union(*layers[1:]))) for t in range(2, len(T)+1) if lt[i]< 1
    ),name="R4a" )
r5a = modelo.addConstrs(
    (It[i,t] == It[i, t-1] + x[i, t] - quicksum(alpha[j][i]*x[j, t] for j in N_reverse[i]) for i in list(set(K3).intersection(set().union(*layers[1:]))) for t in range(2, len(T)+1) if lt[i]>= 1
    ),name="R5a" )

# Restricciones de MOQs

r6 = modelo.addConstrs(
    (x[i,t] >= z1[i,t]*MOQ1[i] for i in K1+K3 for t in range(1, len(T)+1)
     ),name="R6" )
r7 = modelo.addConstrs(
    (y[i,t] >= z2[i,t]*MOQ2[i] for i in K2+K3 for t in range(1, len(T)+1)
     ),name="R7" )

# Restricciones de activacion de variables binarias
r8 = modelo.addConstrs(
    (x[i,t] <= z1[i,t]*42000 for i in K1+K3 for t in range(1, len(T)+1)
     ),name="R8" )
r9 = modelo.addConstrs(
    (y[i,t] <= z2[i,t]*42000 for i in K2+K3 for t in range(1, len(T)+1)
     ),name="R9" )

modelo.update()

In [15]:
# Definición de la función objetivo
modelo.setObjective(quicksum(quicksum(D[t-1][item_indices[i],customer_indices[r]]*B[t-1][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R for i in layers[0]+[39]) 
                             - quicksum(c1[i]*x[i,t] for i in K1+K3)
                             - quicksum(c2[i]*y[i,t] for i in K2+K3)
                             for t in range(1, len(T)+1)), sense = GRB.MAXIMIZE)

In [16]:
# Optimizacion
modelo.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es
Optimize a model with 4160 rows, 40651 columns and 11684 nonzeros
Model fingerprint: 0xa7671361
Variable types: 0 continuous, 40651 integer (38077 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [9e-02, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+04]
Presolve removed 3947 rows and 40425 columns
Presolve time: 0.09s
Presolved: 213 rows, 226 columns, 763 nonzeros
Variable types: 0 continuous, 226 integer (98 binary)
Found heuristic solution: objective 3593034.8257
Found heuristic solution: objective 3629804.4458

Root relaxation: objective 3.661559e+06, 32 iterations, 0.00 seconds (0.00 work units)



In [50]:
# for t in range(1,len(T)+1):
#     for i in K1+K3:
#         if z1[i,t].X!=0:
#             print(i,t)
#             print(x[i,t].X)
            
# for t in range(1,len(T)+1):
#     for i in K2+K3:
#         if z2[i,t].X!=0:
#             print(i,t)
#             print(y[i,t].X)

for t in range(1, len(T) + 1):
    for i in layers[0] + [39]:
        for r in R:
            if w[i, r, t].X != 0:
                print(f"Item, cliente, horizonte temporal: {i, r, t}")
                print(f"Demanda {D[t-1][item_indices[i],customer_indices[r]]}")
                print(f"Stock antes: {It[i,t-1].X}")
                print(f"Stock despues: {It[i,t].X}")
                if i in K1+K3:
                    print(f"Cantidad Producida {x[i,t].X}")
                if i in K2+K3:
                    print(f"Cantidad Comprada {y[i,t].X}")
            else:
                if D[t-1][item_indices[i],customer_indices[r]] != 0:
                    print(f"Item, cliente, horizonte temporal: {i, r, t}")
                    print(f"NO SE SATISFACE")
    print("------------------------------------------")

                        

------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
Item, cliente, horizonte temporal: (3, '101695', 8)
Demanda 23.0
Stock antes: 997.0
Stock despues: 974.0
Cantidad Producida -0.0
Cantidad Comprada -0.0
Item, cliente, horizonte temporal: (23, '101371', 8)
Demanda 270.0
Stock antes: 2183.0
Stock despues: 1638.0
Cantidad Producida -0.0
Cantidad Comprada -0.0
Item, cliente, horizonte temporal: (23, '101706', 8)
Demanda 275.0
Stock antes: 2183.0
Stock despues: 1638.0
Cantidad Producida -0.0
Cantidad Comprada -0.0
Item, cliente, horizonte temporal: (36, '777774', 8)
Demanda 1133.0
Stock antes: 3215.0
Stock despues: 2082.0
Cantidad Producida 0.0
Cantidad Comprada -0.0
Item, cliente, horizonte temporal: (37, '101695', 8)
Demanda 37.0
Stock antes: 

In [51]:
modelo.close()
env.close()